## Utilidades

In [1]:
# Devuelve el exponente lider del polinomio f
def exp(f):
    return f.exponents()[0]

In [2]:
# Devuelve el maximo en cada posición de alpha y beta
def lcm(alpha, beta):
    return alpha.emax(beta)

In [3]:
# Calcula el S-polinomio de f y g en R
def S(R, f, g):
    alpha = exp(f)
    beta = exp(g)
    gamma = lcm(alpha, beta)
    delta1 = gamma.esub(alpha)
    delta2 = gamma.esub(beta)
    return g.lc() * R.monomial(*delta1) * f - f.lc() * R.monomial(*delta2) * g

## Algoritmo de división


In [4]:
# Divide f entre F en R, devuelve los coef y el resto
def division(R, f, F):
    # Polinomio actual
    p = f
    # Resto
    r = 0
    # Nº de polinomios
    s = len(F)
    # Coeficientes (q_i)
    Q = []
    for i in range(s):
        Q.append(0)
    # Hasta que p sea 0
    while p != 0:
        i = 0
        step = 0
        # Si se puede dividir por algun f_i
        while i < s and step == 0:
            # Calculamos gamma (exp(p) = exp(f_i) + gamma)
            gamma = exp(p).esub(exp(F[i]))
            # Gamma debe tener todas las posiciones no negativas
            if all(map(lambda x : x >= 0, gamma)):
                # Coef
                coef = p.lc() / F[i].lc()
                # Actualizamos q_i y p
                Q[i] += coef * R.monomial(*gamma)
                p -= coef * R.monomial(*gamma) * F[i]
                step = 1
            else:
                i += 1
        # Si no es divisible entre ningun f_i añadimos al resto
        if step == 0:
            r += p.lt()
            p -= p.lt()
    return r, Q

Ejemplo:

In [5]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
f = x^2*y+x*y^2+y^2
f1 = x*y-1
f2 = y^2-1
print(division(R, f, [f2, f1]))
print(division(R, f, [f1, f2]))

(2*x + 1, [x + 1, x])
(x + y + 1, [x + y, 1])


## Comprobar si un polinomio pertenece a un ideal

In [6]:
# Si un polinomio f pertenece a un ideal I en R
def check_in_ideal(R, f, I):
    # Otras alternativas
    # -------------------
    # f in I
    # I.reduce(f) == 0
    # -------------------
    # Base de groebner
    B = I.groebner_basis()
    r, _ = division(R, f, B)
    return r == 0

Ejemplo:

In [7]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
f = x^2*y^2 + x^2*y - y + 1
I = R.ideal([x*y^2 + x, x*y-y^3])
print(check_in_ideal(R, f, I))
print(check_in_ideal(R, I.groebner_basis()[0], I))

False
True


## Criterio de Buchberger

In [8]:
# Comprueba si el conjunto de generadores G es base de Groebner en R
def criterio_buchberger(R, G):
    t = len(G)
    for i in range(t):
        for j in range(i + 1, t):
            # G base de Groebner sii rem(S(g_i, g_j), G) = 0
            r, _ = division(R, S(R, G[i], G[j]), G)
            if r != 0:
                return false
    return true

In [9]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
I = R.ideal([x*y^2 + x*y, y^5, y*z^4, x^2*y*z])
print(criterio_buchberger(R, I.groebner_basis()))
print(criterio_buchberger(R, [x*y^2 + x*y, y^5, y*z^4, x^2*y*z]))

True
False


## Algoritmo de Buchberger (obtener base de Groebner)

In [10]:
# Calcula una basa de Groebner para el ideal generado por F en R
def algoritmo_buchberger(R, F):
    G = F
    while true:
        G_prime = G
        for i in range(len(G_prime)):
            for j in range(len(G_prime)):
                if i != j:
                    r, _ = division(R, S(R, G_prime[i], G_prime[j]), G_prime)
                    if r != 0:
                        G.append(r)
        if G_prime == G:
            break
    return G    

In [90]:
R.<x,y> = PolynomialRing(QQ, ["x", "y"], order = "lex")
F = [x^2*y - 1, x*y^2 - x]
B = algoritmo_buchberger(R,F)
print(B)
print(criterio_buchberger(R, B))
print(groebner_reducida(R, B))
print(R.ideal(F).groebner_basis())

[x^2*y - 1, x*y^2 - x, x^2 - y, y^3 - y]
False
[y^2 - 1, x^2 - y]
[x^2 - y, y^2 - 1]


## Base de Groebner reducida

In [105]:
def groebner_reducida(R, G):
    i = 0
    for _ in range(len(G)):
        # rem(g, [G \ {g}])
        r, Q = division(R, G[i], G[:i] + G[i+1:])
        # Si r != 0, lo añadimos
        if r != 0:
            # lc(r) = 1
            r /= r.lc()
            G =  G[:i] + [r] + G[i+1:]
            i += 1
        # Si r == 0, lo descartamos
        else:
            G = G[:i] + G[i+1:]
    return G

In [106]:
R.<x,y,z> = PolynomialRing(QQ, ["x", "y", "z"], order = "lex")
F = [x^2*y - 1, x*y^2 - x]
B = algoritmo_buchberger(R,F)
print(B)
print(groebner_reducida(R,B))
print(R.ideal(F).groebner_basis())

[x^2*y - 1, x*y^2 - x, x^2 - y, y^3 - y]
[y^2 - 1, x^2 - y]
[x^2 - y, y^2 - 1]


In [107]:
R.<x,y,z,w> = PolynomialRing(QQ, ["x", "y", "z", "w"], order = "lex")
F = [3*x-6*y-2*z, 2*x-4*y+4*w, x-2*y-z-w]
B = algoritmo_buchberger(R,F)
print(groebner_reducida(R, B))
print(R.ideal(F).groebner_basis())

[x - 2*y + 2*w, z + 3*w]
[x - 2*y + 2*w, z + 3*w]
